In [2]:
from neuron import *
from encodingsource import *
from hsgs import *
from classical_neuron import *
from classical_pso import *
from sf import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from encodingv2 import *

simulator = Aer.get_backend('qasm_simulator')

/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Imprimindo um circuito gerado pelo Encoding

In [3]:
circuit = QuantumCircuit()
q_input = QuantumRegister(3, 'q_input')
c_output = ClassicalRegister(1, 'c_output')
circuit.add_register(q_input)
circuit.add_register(c_output)

v = [1,1,1,1]
encodingGenerator2(v, circuit, q_input)

circuit.draw()

### Criando neuronio usando hsgs, sf, encoding e comparando com o resultado clássico


In [4]:
def barrar(v):
    for i in range(len(v)):
        v[i] *= -1 

#inputVector = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1]#[1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
#weightVector = [-1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1]#[1, -1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, -1, 1, 1]


inputVector = [-1, 1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1] 
weightVector = [-0.03216409993230598, -0.1520999574184498, 0.11549307734856797, 0.07819667776917735, -0.08683451299393112, 0.07428209202455867, -0.011628350965060036, -0.058852035623669065, 0.02091024109530386, 0.18422984345689053, -0.12001327917718091, -0.17281808213991004, 0.004445115622186578, 0.09949334308845695, 0.0644906589346581, 0.17559849756856852, -0.088707282935652, -0.017908127406289862, -0.1026406028784107, 0.1885141061843412, 0.04853405915763267, -0.1894412214705037, -0.18971428207273894, -0.16080045829897144, -0.14494276618814597, 0.16133137423246743, 0.032410503109816365, 0.1554236889112614, -0.18999964114222914, -0.02773208631445002, -0.1769047447607832, 0.07340800548043382, 0.04977970093090443, -0.18319674721020804, -0.1611156274775103, -0.08041675891466839, -0.10661038807499998, -0.15819257805946796, -0.07129341872229379, 0.17219959076743557, -0.012891323765141861, 0.13884392920174257, -0.03493177735039414, -0.16967932466594843, -0.1344533324680956, 0.07710909951302578, 0.16230727329009179, -0.11811109546183819, 0.05762967585657125, 0.15720292553372192, -0.16562784891930163, -0.07348394951216673, -0.1298264325738957, -0.06989954242058431, 0.1662649564426879, 0.15549686702988463, -0.1345078290406552, 0.07132344671653838, 0.13820247035438069, -0.06455638157173038, 0.1549689794465881, -0.18061152100490427, 0.021178877641755214, -0.15711833018132063]

operator = "encoding-input"
neuron = createNeuron( inputVector, weightVector, operator)
resultadoEncoding = executeNeuron(neuron, simulator, threshold=None)
print("encoding-input", resultadoEncoding)   
operator = "encoding-weight"
neuron = createNeuron( inputVector, weightVector, operator)
resultadoEncoding = executeNeuron(neuron, simulator, threshold=None)
print("encoding-weight", resultadoEncoding)   
operator = "hsgs"
neuron = createNeuron(inputVector, weightVector, operator)
resultadoHSGS = executeNeuron(neuron, simulator, threshold=None)
print("hsgs", resultadoHSGS)

resultadoClassico = runClassicalNeuronReturnProbability(inputVector, weightVector)
print("classico", resultadoClassico)  

encoding-input 0.048095703125
encoding-weight 0.2176513671875
hsgs 0.252197265625
classico 0.21734167312340363


## Vendo que $U_w$ |$\psi_w$> = |m-1>, considerando HSGS e Encoding para gerar U_w. Atenção para o caso do Encoding.

In [ ]:
weightVector = [-1, -1, 1, 1]#,1,1,1,1,1,1,1,1,1,1,1,1]#[0.05, 0.03, -0.5, 0.01]

##### USANDO HSGS PARA U_I e U_W
n=2
circuit = QuantumCircuit()
q_input = QuantumRegister(n, 'q_input')
c_output = ClassicalRegister(n, 'c_output')

circuit.add_register(q_input)
circuit.add_register(c_output)

#Ui
for i in range(n):
    circuit.h(q_input[i])
hsgsGenerator(weightVector, circuit, q_input, n)

#U_w
hsgsGenerator(weightVector, circuit, q_input, 2)

for i in range(n):
    circuit.h(q_input[i])
    circuit.x(q_input[i])

backend = Aer.get_backend('statevector_simulator')
job = execute(circuit, backend)
result = job.result()
outputstate = result.get_statevector(circuit, decimals=3)

print("resultado simulador HSGS",outputstate)

##### USANDO ENCODING PARA U_I e U_W
weightVector = [-1, -1, 1, 1]#,1,1,1,1,1,1,1,1,1,1,1,1]#[0.05, 0.03, -0.5, 0.01]
n=2
circuit = QuantumCircuit()
q_input = QuantumRegister(n, 'q_input')
c_output = ClassicalRegister(n, 'c_output')

circuit.add_register(q_input)
circuit.add_register(c_output)

#Ui
Encoding(circuit, weightVector, q_input)  

#U_w
hsgsGenerator(weightVector, circuit, q_input, 2)

for i in range(n):
    circuit.h(q_input[i])
    circuit.x(q_input[i])

backend = Aer.get_backend('statevector_simulator')
job = execute(circuit, backend)
result = job.result()
outputstate = result.get_statevector(circuit, decimals=3)

print("resultado simulador Encoding",outputstate)


## Executar um teste com 10 entradas diferentes para n=3, com pesos que sejam diferentes nas seguintes quantidades #0 #1, #2, #3, #4, #5, #6, #7, #8, em relação às entradas e comparar o erro do SF, HSGS e Encoding em relação à saída teórica esperada. 

In [ ]:
n=3
dicOldInput = {}

qtdeEntradas = 20
qtdeExperimentoDiferencas = 2**n
repeticoes = 3
precisao = 3

errosMedioHSGS = [0]*qtdeExperimentoDiferencas
errosMedioSF = [0]*qtdeExperimentoDiferencas
errosMedioEncoding = [0]*qtdeExperimentoDiferencas

for i in range(qtdeEntradas):
    print(i, qtdeEntradas)
    procureNovaEntrada = True
    while procureNovaEntrada:
        inputVector = generateRandomState(2**n)
        if not tuple(inputVector) in dicOldInput:
            dicOldInput[tuple(inputVector)] = True
        procureNovaEntrada = False
    for diferencaEntradaPeso in range(qtdeExperimentoDiferencas):
        erroMedioHSGS = 0
        erroMedioSF = 0
        erroMedioEncoding = 0
        for repeticao in range(repeticoes):
            weightVector = generateWeightWithSomeDifference(inputVector, diferencaEntradaPeso)
            operator = "hsgs"
            neuron = createNeuron(inputVector, weightVector, operator)
            resultadoHSGS = executeNeuron(neuron, simulator, threshold=None)
            operator = "sf"
            neuron = createNeuron(inputVector, weightVector, operator)
            resultadoSF =executeNeuron(neuron, simulator, threshold=None)
            operator = "encoding"
            neuron = createNeuron(inputVector, weightVector, operator)
            resultadoEncoding =executeNeuron(neuron, simulator, threshold=None)
            resultadoClassico = runClassicalNeuronReturnProbability(inputVector, weightVector)
            erroHSGS = abs(resultadoHSGS-resultadoClassico)
            erroSF = abs(resultadoSF-resultadoClassico)
            erroEncoding = abs(resultadoEncoding-resultadoClassico)
            
            erroMedioHSGS += round(erroHSGS,precisao)/(repeticoes*qtdeEntradas)
            erroMedioSF += round(erroSF,precisao)/(repeticoes*qtdeEntradas)
            erroEncoding += round(erroEncoding,precisao)/(repeticoes*qtdeEntradas)
        
        errosMedioHSGS[diferencaEntradaPeso] += erroMedioHSGS
        errosMedioSF[diferencaEntradaPeso] += erroMedioSF
        errosMedioEncoding[diferencaEntradaPeso] += erroEncoding

print(errosMedioHSGS,errosMedioSF,errosMedioEncoding )
plt.plot( range(qtdeExperimentoDiferencas), errosMedioHSGS, label="HSGS" )
plt.plot( range(qtdeExperimentoDiferencas), errosMedioSF, label="SF")
plt.plot( range(qtdeExperimentoDiferencas), errosMedioEncoding,label="encoding")
plt.legend()




### Importando a base e separando a classe

### Treinando neuronio clássico e quântico com Regra Delta

In [ ]:
inputVectors = [[-1,1,-1,1,1,1,1,-1],[1,1,-1,1,1,1,1,-1]]
weightVector = [-1,-1,1,1,-1,-1,1,1]
#weightVector = normalize(weightVector)

y_trains = [1,1]
n_epochs = 2000
threshold = 0.5

weightVectorHSGS = weightVector[:]
#weightVectorSF = weightVector[:]
weightVectorEncoding = weightVector[:]

tamTreinamento = len(inputVectors)

for iteration in range(n_epochs):
    print("\niteration", iteration)
    erroHSGS = 0
    erroEncoding = 0
    for posicaoTreinamento in range(tamTreinamento):
        inputVector = inputVectors[posicaoTreinamento]
        y_train = y_trains[posicaoTreinamento]

        """
        executando o HSGS
        """
        operator = "hsgs"
        wBinaryHSGSBinary = deterministicBinarization(weightVectorHSGS) # Binarization of Real weights
        neuron = createNeuron(inputVector, wBinaryHSGSBinary, operator)
        resultadoHSGS = executeNeuron(neuron, simulator, threshold=None)
        print("res hsgs",resultadoHSGS, y_train)

        """
        treinando o HSGS
        """
        deltaRule(inputVector, weightVectorHSGS, threshold=0.5, lr=0.2, y_train=y_train, out=resultadoHSGS)
        #print()

        """
        executando o encoding
        """
        operator = "encoding"
        neuron = createNeuron(inputVector, weightVectorEncoding, operator)
        resultadoEncoding =executeNeuron(neuron, simulator, threshold=None)
        print("res encoding",resultadoEncoding, y_train)
        """
        treinando o encoding
        """
        deltaRule(inputVector, weightVectorEncoding, threshold=0.5, lr=0.2, y_train=y_train, out=resultadoEncoding)

        """
        erros
        """
        resultadoEncoding_bin = 0
        if abs(resultadoEncoding) > threshold:
            resultadoEncoding_bin = 1

        resultadoHSGS_bin = 0
        if abs(resultadoHSGS) > threshold:
            resultadoHSGS_bin = 1

        #print("\nerro Encoding:", abs(resultadoEncoding_bin-y_train))
        #print("erro HSGS:", abs(resultadoHSGS_bin-y_train))
        erroHSGS += abs(resultadoHSGS_bin-y_train)
        erroEncoding += abs(resultadoEncoding_bin-y_train)
    print("erro HSGS", erroHSGS, "erro encoding", erroEncoding)
    print(weightVectorHSGS, wBinaryHSGSBinary,weightVectorEncoding)
    




### Rodando neuronio quântico em dataset com threshold em 0.5

In [ ]:
operator = "hsgs"
weightVector = [-1,1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1]#[1, -1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, -1, 1, 1]

runDataset(listOfInput=x_train,
           listOfExpectedOutput=y_train,
           weightVector=weightVector,
           circuitGeneratorOfUOperator=operator, 
           simulator=simulator, 
           threshold=0.5, 
           memoryOfExecutions={})

### Treinando neuronio Clássico com Regra Delta e pesos reais

In [ ]:
runNeuron (nb_epochs=23,
           binaryWeights=False,
           stochastic=False)

### Treinando neuronio Quântico com Regra Delta

In [ ]:
trainNeuron(method='delta',
            nb_epochs=30,
            binaryWeights=True,
            stochastic=False,
            listOfInput=x_train,
            listOfExpectedOutput=y_train,
            circuitGeneratorOfUOperator='hsgs',
            simulator=simulator,
            threshold=0.5,
            lr = 0.01,
            memoryOfExecutions={})

In [ ]:
trainNeuron(method='delta',
            nb_epochs=30,
            binaryWeights=True,
            stochastic=False,
            listOfInput=x_train,
            listOfExpectedOutput=y_train,
            circuitGeneratorOfUOperator='hsgs',
            simulator=simulator,
            threshold=0.3,
            lr = 0.01,
            memoryOfExecutions={})